In [10]:
import pandas as pd
import random
import shutil
import os

In [2]:
#Read data
df = pd.read_csv('security-data.csv')

#Column names
headers = [ 'id', 'age', 'bmi_per_age_gender', 'bmi_per_age_gender_time',
            'body_height', 'body_height_time', 'body_temperature',
            'body_temperature_time', 'body_weight', 'body_weight_time', 'country',
            'diastolic_blood_pressure', 'diastolic_blood_pressure_time',
            'disease_time', 'erythrocytes_in_blood', 'erythrocytes_in_blood_time',
            'gender', 'heart_rate', 'heart_rate_time', 'heboglobin_time',
            'hemoglobin', 'leukocytes', 'leukocytes_time', 'respiratory_rate',
            'respiratory_rate_time', 'state', 'systolic_blood_pressure',
            'systolic_blood_pressure_time'  ]
df.rename( columns={'Unnamed: 0':'id'}, inplace=True)

#ID numbers
id_nums = random.sample(range(10000000000, 99999999999), df.shape[0])
df.iloc[:,0] = id_nums

df.head()

,id,age,bmi_per_age_gender,bmi_per_age_gender_time,body_height,body_height_time,body_temperature,body_temperature_time,body_weight,body_weight_time,...,heart_rate_time,heboglobin_time,hemoglobin,leukocytes,leukocytes_time,respiratory_rate,respiratory_rate_time,state,systolic_blood_pressure,systolic_blood_pressure_time
0,91371498722,73,0.000,0,154.5,2019,0.000,0,65.9,2019,...,2019,2018,13.173,10.238,2018,15.0,2019,Massachusetts,155,2019
1,31430567041,-2004,0.000,0,0.0,0,37.776,2011,0.0,0,...,0,0,0.000,0.000,0,0.0,0,Massachusetts,0,0
2,60981608420,5,71.555,2018,104.4,2018,38.163,2015,17.6,2018,...,2018,2013,16.372,5.051,2013,12.0,2018,Massachusetts,116,2018
3,70581844200,48,0.000,0,182.3,2017,0.000,0,101.2,2017,...,2017,2015,16.426,10.433,2015,13.0,2017,Massachusetts,125,2017
4,96442875295,44,0.000,0,169.7,2016,0.000,0,87.2,2016,...,2016,0,0.000,0.000,0,14.0,2016,Massachusetts,126,2016


In [3]:
df.loc[:,headers[0:2]]

,id,age
0,91371498722,73
1,31430567041,-2004
2,60981608420,5
3,70581844200,48
4,96442875295,44
...,...,...
12768,69023864008,14
12769,19289889448,3
12770,77875501822,19
12771,41615796031,31


In [11]:
def parse_data(data, splitAmount):
    # ID column
    parsed = [data.loc[:,headers[0]],]
    i=1
    while i < len(headers):
        # Other columns
        parsed.append(data.loc[:,headers[i:i+splitAmount]])
        i+=splitAmount

    return parsed


def save_data(data, splitAmount):
    # Parse
    parsed = parse_data(data, splitAmount) 

    # Create Folder
    folder = '/'+str(splitAmount)
    if os.path.exists(folder):
        shutil.rmtree(folder)
    os.mkdir(folder)
    
    # Save all
    parsed[0].to_csv(folder+'/rep-main.csv', index=False)
    i=1
    for p in parsed[1:]:
        p.to_csv(folder+'/rep-'+str(i)+'.csv', index=False)
        print('Saved to '+folder+'/rep-'+str(i)+'.csv')
        i+=1


In [12]:
save_data(df, 3)
  

Saved to 3/rep-1.csv
Saved to 3/rep-2.csv
Saved to 3/rep-3.csv
Saved to 3/rep-4.csv
Saved to 3/rep-5.csv
Saved to 3/rep-6.csv
Saved to 3/rep-7.csv
Saved to 3/rep-8.csv
Saved to 3/rep-9.csv
